# viz-4a-anomaly-detection.ipynb

The first notebook in this set of examples (viz-4x) demonstrates how to replicate one of the custom charts of Kibana's Machine Learning plugin as an embeddable chart for dashboards.

In [1]:
import datetime
import altair as alt
import eland as ed
import json
import numpy as np
import matplotlib.pyplot as plt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
index_name = '.ml-anomalies-fq_single_count_15m'
vis_name = 'ml-anomalies-fq'

In [3]:
ed_df = ed.DataFrame('localhost:9200', index_name)
ed_df_number = ed_df.select_dtypes(include=np.number)
ed_df_number.shape

(976, 66)

In [4]:
url = 'http://localhost:9200/'+index_name+'/_search?size=10000'
url_data = alt.Data(url=url, format=alt.DataFormat(property='hits.hits',type='json'))

fields = [
    'anomaly_score',
    'actual',
    'typical',
    'event_count',
    'model_lower',
    'model_median',
    'model_upper',
    'timestamp',
    'result_type'
]

rename_dict = dict((a, 'datum._source.'+a) for a in fields)

base = alt.Chart(url_data).transform_calculate(**rename_dict)

url_chart = base.transform_fold(
    fields,
    as_=['attribute', 'value']
).mark_bar().encode(
    alt.X('value:Q', bin=True, title=''),
    alt.Y('count()', title=''),
    tooltip=[
        alt.Tooltip('value:Q', bin=True, title='x'),
        alt.Tooltip('count()', title='y')
    ]
).properties(
    width=100,
    height=100
)

url_charts = alt.ConcatChart(
    concat=[
      url_chart.transform_filter(alt.datum.attribute == attribute).properties(title=attribute)
      for attribute in sorted(fields)
    ],
    columns=4
).resolve_axis(
    x='independent',
    y='independent'
).resolve_scale(
    x='independent', 
    y='independent'
)

url_charts

alt.ConcatChart(...)

In [5]:
# custom: ['#1f77b4', 'lightgray', 'gray']
# ml:     ['#32a7c2', '#c8e6ef', '#bbd7df']

colors = {'model_bounds':'lightgray','model_median':'gray','actual':'#1f77b4'}

base = alt.Chart(url_data, width=800, height=300).transform_calculate(**rename_dict)

model_bounds = base.mark_area(color=colors['model_bounds'], opacity=0.5).transform_filter(
    alt.datum.result_type == 'model_plot'
).encode(
    alt.X('timestamp:T', scale=alt.Scale(zero=False,nice=False), title='', axis=alt.Axis(grid=False)),
    alt.Y('mean(model_upper):Q'),
    alt.Y2('mean(model_lower):Q'),
)

model_median = base.mark_line(color=colors['model_median'], opacity=0.5, strokeJoin='round').transform_filter(
    alt.datum.result_type == 'model_plot'
).encode(
    alt.X('timestamp:T', scale=alt.Scale(zero=False,nice=False), title='', axis=alt.Axis(grid=False)),
    alt.Y('mean(model_median):Q', title='')
)

actual = base.mark_line(color=colors['actual'], strokeJoin='round').transform_filter(
    alt.datum.result_type == 'model_plot'
).encode(
    alt.X('timestamp:T', scale=alt.Scale(zero=False,nice=False), title='', axis=alt.Axis(grid=False)),
    alt.Y('mean(actual):Q')
)

anomalies = base.mark_point(color='red', size=60).transform_aggregate(
    mean_actual='mean(actual)',
    mean_anomaly_score='mean(anomaly_score)',
    groupby=["timestamp"]
).transform_filter(
    alt.datum.mean_anomaly_score > 0
).encode(
    alt.X('timestamp:T', scale=alt.Scale(zero=False,nice=False), title='', axis=alt.Axis(grid=False)),
    alt.Y('mean_actual:Q')
)

tooltip = base.mark_circle(opacity=0, size=100).transform_aggregate(
    mean_actual='mean(actual)',
    mean_anomaly_score='mean(anomaly_score)',
    groupby=["timestamp"]
).encode(
    alt.X('timestamp:T', scale=alt.Scale(zero=False,nice=False), title='', axis=alt.Axis(grid=False)),
    alt.Y('mean_actual:Q'),
    tooltip=['mean_actual:Q', 'mean_anomaly_score:Q', 'timestamp:T']
)

chart = (model_bounds + model_median + actual + anomalies + tooltip).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=14,
    fontWeight='bold',
    anchor='start',
    color='gray'
)

chart

alt.LayerChart(...)

In [6]:
from kibana_vega_util import saveVegaLiteVis
from elasticsearch import Elasticsearch 
es=Elasticsearch([{'host':'localhost','port':9200}])

saveVegaLiteVis(es, index_name, vis_name, chart, resultSize=1000)

{'_index': '.kibana_3',
 '_id': 'visualization:ml-anomalies-fq',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 439,
 '_primary_term': 2}